In [1]:
import pandas as pd
import numpy as np


In [3]:
df = pd.read_csv('../data/processed/merged_cleaned_weather_data.csv')
df.head()

,DATE,REPORT_TYPE,wind_speed_raw,wind_dir_sin,wind_dir_cos,ceiling_coverage,visibility_m,temperature_C,SLP_hpa,DEW_C,MA1_main,MA1_sec,GA1_amt,GA1_height,GA1_type,MD1_m1,MD1_m2
0,2019-01-01 00:00:00,FM-12+FM-15,28.0,-0.866025,-0.500000,0.0,6000.0,0.05,1030.2,-0.95,10280.0,9996.0,4.5,720.0,6.0,8.0,11.0
1,2019-01-01 01:00:00,FM-12+FM-15,30.5,-0.642788,-0.766044,0.0,6000.0,0.90,1029.9,-0.90,10280.0,9994.0,7.0,705.0,6.0,7.0,11.0
2,2019-01-01 02:00:00,FM-12+FM-15,35.5,-0.571394,-0.816035,0.0,6000.0,0.90,1028.6,-1.00,10270.0,9981.0,5.0,750.5,6.0,8.0,21.0
3,2019-01-01 03:00:00,FM-12+FM-15,25.5,-0.704416,-0.704416,0.0,5000.0,-0.90,1027.3,-1.90,10250.0,9967.0,NaN,NaN,2.0,8.0,29.0
4,2019-01-01 04:00:00,FM-12+FM-15,28.0,-0.571394,-0.816035,0.0,5000.0,-1.00,1026.3,-2.15,10240.0,9957.0,NaN,NaN,2.0,6.0,37.0


In [4]:
df.isna().sum()

DATE                    0
REPORT_TYPE             0
wind_speed_raw         18
wind_dir_sin         3847
wind_dir_cos         3847
ceiling_coverage       59
visibility_m           91
temperature_C           1
SLP_hpa               171
DEW_C                   0
MA1_main              181
MA1_sec               142
GA1_amt              9815
GA1_height          15769
GA1_type             5345
MD1_m1               2581
MD1_m2                117
dtype: int64

In [7]:
from sklearn.impute import KNNImputer

df['DATE'] = pd.to_datetime(df['DATE'])
imputer = KNNImputer(n_neighbors=3)
numeric_cols = df.select_dtypes(include=[np.number]).columns
df[numeric_cols] = imputer.fit_transform(df[numeric_cols])

# mozna jakos inaczej ale mi sie nie chce
# jak zostawiamy imputer to wazne zeby go zapisac np jako pickle i potem dokladnie ten sam uzywac na testowych danych


In [8]:
df = df.drop(columns=['REPORT_TYPE'])

In [9]:
df['year'] = df['DATE'].dt.year
df['month'] = df['DATE'].dt.month
df['dayofyear'] = df['DATE'].dt.dayofyear
df['dayofweek'] = df['DATE'].dt.dayofweek
df['day'] = df['DATE'].dt.day
df['hour'] = df['DATE'].dt.hour


In [10]:
df = df.drop(columns=['DATE'])

In [21]:
df.columns

Index(['wind_speed_raw', 'wind_dir_sin', 'wind_dir_cos', 'ceiling_coverage',
       'visibility_m', 'temperature_C', 'SLP_hpa', 'DEW_C', 'MA1_main',
       'MA1_sec', 'GA1_amt', 'GA1_height', 'GA1_type', 'MD1_m1', 'MD1_m2',
       'year', 'month', 'dayofyear', 'dayofweek', 'day', 'hour'],
      dtype='object')

In [11]:
df.head()

,wind_speed_raw,wind_dir_sin,wind_dir_cos,ceiling_coverage,visibility_m,temperature_C,SLP_hpa,DEW_C,MA1_main,MA1_sec,...,GA1_height,GA1_type,MD1_m1,MD1_m2,year,month,dayofyear,dayofweek,day,hour
0,28.0,-0.866025,-0.500000,0.0,6000.0,0.05,1030.2,-0.95,10280.0,9996.0,...,720.000000,6.0,8.0,11.0,2019,1,1,1,1,0
1,30.5,-0.642788,-0.766044,0.0,6000.0,0.90,1029.9,-0.90,10280.0,9994.0,...,705.000000,6.0,7.0,11.0,2019,1,1,1,1,1
2,35.5,-0.571394,-0.816035,0.0,6000.0,0.90,1028.6,-1.00,10270.0,9981.0,...,750.500000,6.0,8.0,21.0,2019,1,1,1,1,2
3,25.5,-0.704416,-0.704416,0.0,5000.0,-0.90,1027.3,-1.90,10250.0,9967.0,...,572.833333,2.0,8.0,29.0,2019,1,1,1,1,3
4,28.0,-0.571394,-0.816035,0.0,5000.0,-1.00,1026.3,-2.15,10240.0,9957.0,...,777.500000,2.0,6.0,37.0,2019,1,1,1,1,4


In [15]:
df_2019 = pd.read_excel('../data/raw/AirQuality_Krakow/2019_PM10_1g.xlsx')
df_2020 = pd.read_excel('../data/raw/AirQuality_Krakow/2020_PM10_1g.xlsx')
df_2021 = pd.read_excel('../data/raw/AirQuality_Krakow/2021_PM10_1g.xlsx')
df_2022 = pd.read_excel('../data/raw/AirQuality_Krakow/2022_PM10_1g.xlsx')
df_2023 = pd.read_excel('../data/raw/AirQuality_Krakow/2023_PM10_1g.xlsx')

df_all = pd.concat([df_2019, df_2020, df_2021, df_2022, df_2023], ignore_index=True)

In [19]:
station_cols = [col for col in df_all.columns if col.startswith('MpKrak')]
df_melted = df_all.melt(
    id_vars=['DateTime'],
    value_vars=station_cols,
    var_name='station',
    value_name='pm10_value'
)

In [20]:
df_melted.head()

,DateTime,station,pm10_value
0,2019-01-01 01:00:00,MpKrakAlKras,88.2185
1,2019-01-01 02:00:00,MpKrakAlKras,95.2209
2,2019-01-01 03:00:00,MpKrakAlKras,85.3689
3,2019-01-01 04:00:00,MpKrakAlKras,70.8575
4,2019-01-01 05:00:00,MpKrakAlKras,50.1935


In [ ]:
df_weather = df.copy()
df_weather['DateTime'] = pd.to_datetime(df_weather[['year', 'month', 'day', 'hour']])

df_melted['DateTime'] = pd.to_datetime(df_melted['DateTime'])

df_final = df_melted.merge(
    df_weather, 
    on='DateTime', 
    how='left'
)

In [23]:
df_final

,DateTime,station,pm10_value,wind_speed_raw,wind_dir_sin,wind_dir_cos,ceiling_coverage,visibility_m,temperature_C,SLP_hpa,...,GA1_height,GA1_type,MD1_m1,MD1_m2,year,month,dayofyear,dayofweek,day,hour
0,2019-01-01 01:00:00,MpKrakAlKras,88.2185,30.5,-0.642788,-0.766044,0.0,6000.0,0.90,1029.9,...,705.000000,6.0,7.000000,11.0,2019.0,1.0,1.0,1.0,1.0,1.0
1,2019-01-01 02:00:00,MpKrakAlKras,95.2209,35.5,-0.571394,-0.816035,0.0,6000.0,0.90,1028.6,...,750.500000,6.0,8.000000,21.0,2019.0,1.0,1.0,1.0,1.0,2.0
2,2019-01-01 03:00:00,MpKrakAlKras,85.3689,25.5,-0.704416,-0.704416,0.0,5000.0,-0.90,1027.3,...,572.833333,2.0,8.000000,29.0,2019.0,1.0,1.0,1.0,1.0,3.0
3,2019-01-01 04:00:00,MpKrakAlKras,70.8575,28.0,-0.571394,-0.816035,0.0,5000.0,-1.00,1026.3,...,777.500000,2.0,6.000000,37.0,2019.0,1.0,1.0,1.0,1.0,4.0
4,2019-01-01 05:00:00,MpKrakAlKras,50.1935,30.5,-0.766044,-0.642788,0.0,6000.0,-0.65,1025.0,...,1280.500000,6.0,7.000000,36.0,2019.0,1.0,1.0,1.0,1.0,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
306763,2023-12-31 20:00:00,MpKrakZloRog,32.3285,28.0,0.173648,0.984808,0.5,15000.0,3.05,1009.0,...,1144.333333,3.0,7.000000,17.0,2023.0,12.0,365.0,6.0,31.0,20.0
306764,2023-12-31 21:00:00,MpKrakZloRog,38.2589,20.5,0.336824,0.925417,0.5,10000.0,2.10,1009.1,...,1103.833333,2.0,6.000000,11.0,2023.0,12.0,365.0,6.0,31.0,21.0
306765,2023-12-31 22:00:00,MpKrakZloRog,41.0132,17.5,0.816035,0.571394,0.0,7000.0,2.70,1009.4,...,1308.833333,2.0,5.000000,1.0,2023.0,12.0,365.0,6.0,31.0,22.0
306766,2023-12-31 23:00:00,MpKrakZloRog,47.8850,12.5,0.992404,-0.086824,0.0,4000.0,2.00,1009.3,...,713.833333,2.0,6.666667,2.0,2023.0,12.0,365.0,6.0,31.0,23.0


In [24]:
df_final.columns

Index(['DateTime', 'station', 'pm10_value', 'wind_speed_raw', 'wind_dir_sin',
       'wind_dir_cos', 'ceiling_coverage', 'visibility_m', 'temperature_C',
       'SLP_hpa', 'DEW_C', 'MA1_main', 'MA1_sec', 'GA1_amt', 'GA1_height',
       'GA1_type', 'MD1_m1', 'MD1_m2', 'year', 'month', 'dayofyear',
       'dayofweek', 'day', 'hour'],
      dtype='object')

In [27]:
!pip install torch

  Using cached setuptools-80.9.0-py3-none-any.whl.metadata (6.6 kB)
  Using cached sympy-1.14.0-py3-none-any.whl.metadata (12 kB)
  Using cached jinja2-3.1.6-py3-none-any.whl.metadata (2.9 kB)
  Using cached MarkupSafe-3.0.2-cp312-cp312-macosx_11_0_arm64.whl.metadata (4.0 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 MB 34.5 MB/s  0:00:02 eta 0:00:01
Using cached sympy-1.14.0-py3-none-any.whl (6.3 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.2/536.2 kB 20.7 MB/s  0:00:00
Using cached jinja2-3.1.6-py3-none-any.whl (134 kB)
Using cached MarkupSafe-3.0.2-cp312-cp312-macosx_11_0_arm64.whl (12 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 45.8 MB/s  0:00:00
Using cached setuptools-80.9.0-py3-none-any.whl (1.2 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9/9 [torch]32m8/9 [torch]]x]s]


In [28]:
import pandas as pd
import numpy as np
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [29]:
class PM10Dataset(Dataset):
    def __init__(self, data, scaler=None, fit_scaler=True):
        self.data = data
        if scaler is None:
            self.scaler = StandardScaler()
        else:
            self.scaler = scaler
            
        if fit_scaler:
            self.scaler.fit(data[['pm10_value', 'wind_speed_raw', 'temperature_C', 'SLP_hpa', 'DEW_C']])
        
        self.scaled_data = self.scaler.transform(data[['pm10_value', 'wind_speed_raw', 'temperature_C', 'SLP_hpa', 'DEW_C']])
        self.sequences = self._create_sequences()
    
    def _create_sequences(self):
        sequences = []
        stations = self.data['station'].unique()
        
        for station in stations:
            station_data = self.data[self.data['station'] == station].sort_values('DateTime')
            station_scaled = self.scaled_data[self.data['station'] == station]
            
            for i in range(2, len(station_data) - 24):
                history_indices = [i-2, i-1] 
                history_features = station_scaled[history_indices]  
                
                target_pm10 = station_scaled[i:i+24, 0] 
                
                hour = station_data.iloc[i]['hour'] / 23.0
                
                sequences.append({
                    'history': history_features,
                    'hour': hour,
                    'target': target_pm10
                })
        
        return sequences
    
    def __len__(self):
        return len(self.sequences)
    
    def __getitem__(self, idx):
        seq = self.sequences[idx]
        return (
            torch.FloatTensor(seq['history']),
            torch.FloatTensor([seq['hour']]),
            torch.FloatTensor(seq['target'])
        )

In [30]:
class PM10LSTMModel(nn.Module):
    def __init__(self, input_size=5, hidden_size=128, num_layers=2, dropout=0.2):
        super(PM10LSTMModel, self).__init__()
        
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, 
                           batch_first=True, dropout=dropout)
        
        self.hour_embedding = nn.Linear(1, 16)
        
        self.fc_layers = nn.Sequential(
            nn.Linear(hidden_size + 16, 256),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(128, 24)  
        )
        
    def forward(self, history, hour):
        lstm_out, (hidden, cell) = self.lstm(history)
        last_hidden = lstm_out[:, -1, :] 
        
        hour_emb = torch.relu(self.hour_embedding(hour))
        
        combined = torch.cat([last_hidden, hour_emb], dim=1)
        
        predictions = self.fc_layers(combined)
        return predictions

def prepare_data(df, test_size=0.2, batch_size=32):
    train_df, val_df = train_test_split(df, test_size=test_size, random_state=42)
    
    train_dataset = PM10Dataset(train_df, fit_scaler=True)
    val_dataset = PM10Dataset(val_df, scaler=train_dataset.scaler, fit_scaler=False)
    
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    
    return train_loader, val_loader, train_dataset.scaler

def train_model(model, train_loader, val_loader, epochs=50, lr=0.001):
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=5, factor=0.5)
    
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    
    train_losses = []
    val_losses = []
    
    for epoch in range(epochs):
        model.train()
        train_loss = 0.0
        
        for history, hour, target in train_loader:
            history, hour, target = history.to(device), hour.to(device), target.to(device)
            
            optimizer.zero_grad()
            outputs = model(history, hour)
            loss = criterion(outputs, target)
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item()
        
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for history, hour, target in val_loader:
                history, hour, target = history.to(device), hour.to(device), target.to(device)
                outputs = model(history, hour)
                loss = criterion(outputs, target)
                val_loss += loss.item()
        
        train_loss /= len(train_loader)
        val_loss /= len(val_loader)
        
        train_losses.append(train_loss)
        val_losses.append(val_loss)
        
        scheduler.step(val_loss)
        
        if epoch % 10 == 0:
            print(f'Epoch {epoch}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}')
    
    return train_losses, val_losses

def predict_for_test_case(model, scaler, history_data, hour, device):
    model.eval()
    
    scaled_history = scaler.transform(history_data)
    history_tensor = torch.FloatTensor(scaled_history).unsqueeze(0).to(device)
    hour_tensor = torch.FloatTensor([[hour / 23.0]]).to(device)
    
    with torch.no_grad():
        predictions = model(history_tensor, hour_tensor)
        
    predictions_np = predictions.cpu().numpy().flatten()
    
    dummy_features = np.zeros((24, 5))
    dummy_features[:, 0] = predictions_np
    
    denormalized = scaler.inverse_transform(dummy_features)
    pm10_predictions = denormalized[:, 0]
    
    return pm10_predictions

In [33]:
df_final = df_final.dropna().reset_index(drop=True)

In [ ]:
df_final['DateTime'] = pd.to_datetime(df_final['DateTime'])

train_loader, val_loader, scaler = prepare_data(df_final)

model = PM10LSTMModel()
train_losses, val_losses = train_model(model, train_loader, val_loader)